# Introduction
The code below demonstrates a 3-D uniaxial strain problem using simple displacement or traction boundary conditions. The location matrix, or element connectivity matrix, is hard-coded for two trilinear hexahedral finite elements (FE), and the application of boundary conditions is similarly inflexible.

## A primer on `np.einsum`
This code makes heavy use of NumPy's `einsum` function. The advantage to using this function is (i) readability (we can use index notation from our notes) and (ii) performance (highly optimized in `c`).

We begin with a few examples of how it works. Full documentation can be found at https://numpy.org/doc/stable/reference/generated/numpy.einsum.html.

In [1]:
import numpy as np

In [2]:
np.set_printoptions(precision=5)

In [3]:
array_1 = np.random.randn(3,3)

In [4]:
print(array_1)

[[-0.61616  0.74639  0.84084]
 [-0.47052  0.12218 -0.1582 ]
 [-1.72667  0.35648 -2.18752]]


First, take the trace of this array, i.e., $A_{ii}$:

In [5]:
def loop_trace(arr):
    trace = 0
    for i in range(arr.shape[0]):
        trace += arr[i,i]
    return trace

In [6]:
def numpy_trace(arr):
    return np.trace(arr)

In [7]:
def einsum_trace(arr):
    return np.einsum('ii', arr)

In [8]:
%timeit loop_trace(array_1)
%timeit numpy_trace(array_1)
%timeit einsum_trace(array_1)

1.15 µs ± 37.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
3.41 µs ± 123 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.05 µs ± 53.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [9]:
print(loop_trace(array_1) == numpy_trace(array_1))
print(loop_trace(array_1) == einsum_trace(array_1))

True
True


Not much difference between these three as far as the timing goes. What about other operations, e.g., $\mathbf{a}\otimes\mathbf{b} = \mathbf{C}$ or $a_ib_j = C_{ij}$?

In [51]:
vec_1   = np.random.randn(3)
vec_2   = np.random.randn(2)
array_2 = np.random.randn(3,2)

In [52]:
def loop_outer(v1, v2):
    outer = np.zeros((v1.shape[0], v2.shape[0]))
    for i in range(v1.shape[0]):
        for j in range(v2.shape[0]):
            outer[i,j] = v1[i]*v2[j]
    return outer

In [53]:
def numpy_outer(v1, v2):
    return np.tensordot(v1, v2, axes=0)

In [54]:
def einsum_outer(v1, v2):
    return np.einsum('i, j -> ij', v1, v2)

In [55]:
%timeit loop_outer(vec_1, vec_2)
%timeit numpy_outer(vec_1, vec_2)
%timeit einsum_outer(vec_1, vec_2)

3.32 µs ± 73 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
12.2 µs ± 910 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.82 µs ± 19.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [15]:
print(loop_outer(vec_1, vec_2) == numpy_outer(vec_1, vec_2))
print(loop_outer(vec_1, vec_2) == einsum_outer(vec_1, vec_2))

[[ True  True]
 [ True  True]
 [ True  True]]
[[ True  True]
 [ True  True]
 [ True  True]]


A bit of a performance boost over a for-loop and well beyond `tensordot`.

Next, do matrix-vector multiplication, e.g., $\mathbf{A}\mathbf{b} = \mathbf{c}$ or $A_{ij}b_j = c_i$:

In [56]:
def loop_matvec(mat, vec):
    vector = np.zeros((mat.shape[0]))
    for i in range(mat.shape[0]):
        for j in range(mat.shape[1]):
            vector[i] += mat[i,j]*vec[j]
    return vector

In [57]:
def numpy_matvec(mat, vec):
    return mat@vec.T

In [58]:
def numpy_matvec_dot(mat, vec):
    return np.dot(mat, vec)

In [59]:
def numpy_matvec_tensordot(mat, vec):
    return np.tensordot(mat, vec, axes=1)

In [60]:
def einsum_matvec(mat, vec):
    return np.einsum('ij, j -> i', mat, vec)

In [61]:
%timeit loop_matvec(array_1, vec_1)
%timeit numpy_matvec(array_1, vec_1)
%timeit numpy_matvec_dot(array_1, vec_1)
%timeit numpy_matvec_tensordot(array_1, vec_1)
%timeit einsum_matvec(array_1, vec_1)

5.72 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
936 ns ± 10.5 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
742 ns ± 16.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
13.5 µs ± 827 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
1.87 µs ± 17.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [22]:
print(loop_matvec(array_1, vec_1))
print(numpy_matvec(array_1, vec_1))
print(numpy_matvec_dot(array_1, vec_1))
print(numpy_matvec_tensordot(array_1, vec_1))
print(einsum_matvec(array_1, vec_1))

[-0.77695  0.2813   3.21222]
[-0.77695  0.2813   3.21222]
[-0.77695  0.2813   3.21222]
[-0.77695  0.2813   3.21222]
[-0.77695  0.2813   3.21222]


We get beat by the traditional `dot` or `@`, which aren't really clear on what the operation taking place under the hood is.

Next, do matrix-matrix multiplcation, e.g., $\mathbf{A}\mathbf{B} = \mathbf{C}$ or $A_{ik}B_{kj} = C_{ij}$

In [23]:
def loop_matmat(mat1, mat2):
    matrix = np.zeros((mat1.shape[0], mat2.shape[1]))
    for i in range(mat1.shape[0]):
        for j in range(mat2.shape[1]):
            for k in range(mat1.shape[1]):
                matrix[i,j] += mat1[i,k]*mat2[k,j]
    return matrix

In [24]:
def numpy_matmat(mat1, mat2):
    return mat1@mat2

In [25]:
def numpy_matmat_dot(mat1, mat2):
    return np.dot(mat1, mat2)

In [26]:
def numpy_matmat_tensordot(mat1, mat2):
    return np.tensordot(mat1, mat2, axes=1)

In [27]:
def einsum_matmat(mat1, mat2):
    return np.einsum('ik, kj -> ij', mat1, mat2)

In [28]:
%timeit loop_matmat(array_1, array_2)
%timeit numpy_matmat(array_1, array_2)
%timeit numpy_matmat_dot(array_1, array_2)
%timeit numpy_matmat_tensordot(array_1, array_2)
%timeit einsum_matmat(array_1, array_2)

16.8 µs ± 402 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
1.18 µs ± 12.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.03 µs ± 11.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
15.6 µs ± 353 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.38 µs ± 74.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [29]:
print(loop_matmat(array_1, array_2))
print(numpy_matmat(array_1, array_2))
print(numpy_matmat_dot(array_1, array_2))
print(numpy_matmat_tensordot(array_1, array_2))
print(einsum_matmat(array_1, array_2))

[[ 0.53027 -0.00642]
 [-0.11208  0.17214]
 [-2.02117  1.25003]]
[[ 0.53027 -0.00642]
 [-0.11208  0.17214]
 [-2.02117  1.25003]]
[[ 0.53027 -0.00642]
 [-0.11208  0.17214]
 [-2.02117  1.25003]]
[[ 0.53027 -0.00642]
 [-0.11208  0.17214]
 [-2.02117  1.25003]]
[[ 0.53027 -0.00642]
 [-0.11208  0.17214]
 [-2.02117  1.25003]]


Here we see that `einsum` is much faster than `tensordot` and the for-loop, and still only slightly behind the traditional methods. But what if we have much higher order tensors, e.g., 4th order?

In [66]:
fourth_order_tensor = np.random.randn(3,3,3,3)
second_order_tensor = np.random.randn(3,3)

In [67]:
def loop_matmat_ho(mat1, mat2):
    matrix = np.zeros((mat1.shape[0], mat1.shape[1]))
    for i in range(mat1.shape[0]):
        for j in range(mat1.shape[1]):
            for k in range(mat1.shape[2]):
                for l in range(mat1.shape[3]):
                    matrix[i,j] += mat1[i,j,k,l]*mat2[k,l]
    return matrix

In [76]:
def numpy_matmat_tensordot_ho(mat1, mat2):
    return np.tensordot(mat1, mat2, axes=2)

In [77]:
def einsum_matmat_ho(mat1, mat2):
    return np.einsum('ijkl, kl -> ij', mat1, mat2)

In [80]:
%timeit loop_matmat_ho(fourth_order_tensor, second_order_tensor)
%timeit numpy_matmat(fourth_order_tensor, second_order_tensor)
%timeit numpy_matmat_dot(fourth_order_tensor, second_order_tensor)
%timeit numpy_matmat_tensordot_ho(fourth_order_tensor, second_order_tensor)
%timeit einsum_matmat_ho(fourth_order_tensor, second_order_tensor)

57.9 µs ± 1.96 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


TypeError: _sum() got an unexpected keyword argument 'axes'

In [78]:
print(loop_matmat_ho(fourth_order_tensor, second_order_tensor))
print(numpy_matmat(fourth_order_tensor, second_order_tensor))
print(numpy_matmat_dot(fourth_order_tensor, second_order_tensor))
print(numpy_matmat_tensordot_ho(fourth_order_tensor, second_order_tensor))
print(einsum_matmat_ho(fourth_order_tensor, second_order_tensor))

[[-0.49064 -0.92841 -0.6752 ]
 [ 0.05969 -2.0381  -1.26636]
 [ 1.25196  2.25958  3.13605]]
[[[[-1.68004e+00 -1.11987e+00 -5.99472e-01]
   [ 2.14177e+00  1.42352e+00  6.36986e-01]
   [ 1.21567e+00  1.06600e+00  2.94696e-01]]

  [[ 2.20018e+00  2.61847e+00  6.73414e-01]
   [ 6.27217e-04  3.61013e-01 -1.05371e-01]
   [ 4.52854e-02  6.53617e-01 -4.76358e-02]]

  [[ 3.51408e-01  7.78784e-01  8.90111e-02]
   [-2.03832e+00 -9.98640e-01 -5.99839e-01]
   [-3.46411e-01 -8.35794e-01 -8.72074e-02]]]


 [[[ 1.23130e+00  3.34061e-01  2.85493e-01]
   [ 1.01805e+00  5.68650e-01  3.62853e-01]
   [-1.28428e+00 -8.55233e-02 -3.78561e-01]]

  [[-4.93035e+00 -5.25141e+00 -1.22678e+00]
   [ 8.91887e-01  3.91188e-01  1.31335e-01]
   [ 4.15474e-01  1.09895e+00  1.29021e-01]]

  [[ 7.73294e-01 -2.19560e-01  3.31158e-02]
   [-1.73852e+00 -1.15922e+00 -6.08298e-01]
   [ 1.85055e-01  1.13555e-01 -4.61360e-02]]]


 [[[-1.46453e+00 -4.76118e-01 -3.46122e-01]
   [-3.28298e+00 -3.04976e+00 -9.89260e-01]
   [ 2.48065e

When we naively used the built-in functions and we got the wrong result. What if we catch this?

In [86]:
print(np.sum(numpy_matmat(fourth_order_tensor, second_order_tensor), axis=(2,3)))
print(np.sum(numpy_matmat_dot(fourth_order_tensor, second_order_tensor), axis=(2,3)))

[[ 3.37924  6.3996  -3.68701]
 [ 2.05204 -8.35068 -2.66671]
 [-4.9518  14.52335  2.31794]]
[[ 3.37924  6.3996  -3.68701]
 [ 2.05204 -8.35068 -2.66671]
 [-4.9518  14.52335  2.31794]]


# 3-D Hexahedral Elements
The methodology of the Python code is to create an object that can be used to represent the finite element.

First, let us create the coordinates of one trilinear hexahedral element:

In [30]:
numEl    = 1
numGauss = 8
numDim   = 3

coordinates = np.zeros((numEl, numGauss, numDim))
coordinates[0,:,:] = np.array([[0.0,  0.0,  0.0],
                               [0.01, 0.0,  0.0],
                               [0.01, 0.01, 0.0],
                               [0.0,  0.01, 0.0],
                               [0.0,  0.0,  0.05],
                               [0.01, 0.0,  0.05],
                               [0.01, 0.01, 0.05],
                               [0.0,  0.01, 0.05]])

With the coordinates, we can now initialize the element object.

In [31]:
class Parameters:
    
    def __init__(self):
        self.float_dtype = np.float64 # argument for precision of NumPy arrays used in element object

In [32]:
import classElement

In [33]:
params  = Parameters()
element = classElement.Element(a_GaussOrder=2, a_ID=0)
element.set_Gauss_Points(params)
element.set_Gauss_Weights(params)
element.set_Coordinates(coordinates[element.ID,:,:])

In [34]:
print(element.coordinates)

[[0.   0.   0.  ]
 [0.01 0.   0.  ]
 [0.01 0.01 0.  ]
 [0.   0.01 0.  ]
 [0.   0.   0.05]
 [0.01 0.   0.05]
 [0.01 0.01 0.05]
 [0.   0.01 0.05]]


Next, we can initialize the shape functions. The displacement shape functions are constant, i.e., we only need to define them once because the local coordinates are not changing. However, the strain-displacement shape functions are not constant because they depend on some deformation at time $t_n$.

Here, we introduce the concept of broadcasting. Broadcasting is an efficient method to perform calculations on large datasets. In this example, our "large" dataset is the 8 Gauss points. Even though that is relatively small, remember that Python does not perform well when using loops. If we were to use higher-order interpolations, looping over the Gauss points to reinitialize the shape functions in memory would become prohibitively expensive.

So the leading dimension of all element-related quantities, whether they be interpolation functions or stress measures (e.g., $\boldsymbol{\sigma}^e$), represents the Gauss point, and trailing dimensions represent the physical interpreation.

In [35]:
print(element.points)
print(element.points.shape)

[[-0.57735 -0.57735 -0.57735]
 [ 0.57735 -0.57735 -0.57735]
 [ 0.57735  0.57735 -0.57735]
 [-0.57735  0.57735 -0.57735]
 [-0.57735 -0.57735  0.57735]
 [ 0.57735 -0.57735  0.57735]
 [ 0.57735  0.57735  0.57735]
 [-0.57735  0.57735  0.57735]]
(8, 3)


$\boldsymbol{\xi}^e = [\xi \quad \eta \quad \zeta]^e$ has shape 8x3 (8 Gauss points, 3 dimensions xyz) 

In [43]:
element.evaluate_Shape_Functions(params)

In [42]:
print(element.xi); print(element.xi.shape)
print(element.eta); print(element.eta.shape)
print(element.zeta); print(element.zeta.shape)

[-0.57735  0.57735  0.57735 -0.57735 -0.57735  0.57735  0.57735 -0.57735]
(8,)
[-0.57735 -0.57735  0.57735  0.57735 -0.57735 -0.57735  0.57735  0.57735]
(8,)
[-0.57735 -0.57735 -0.57735 -0.57735  0.57735  0.57735  0.57735  0.57735]
(8,)


Here, `Nu` is an 8x3x24, i.e.,

$\mathbf{N}^{u,e} := \sum_{a=1}^8 \underbrace{N^u_a(\boldsymbol{\xi})}_{n_{dim}\times(n_{dim}\times n_{Gauss})} = \sum_{a=1}^8 \underbrace{N_a(\boldsymbol{\xi})}_{3\times 24}$

In [45]:
print(element.Nu.shape)

(8, 3, 24)


```python
def evaluate_Shape_Functions(self, Parameters):
        # Initialize the shape functions used for interpolation.        
        #--------------------------------
        # Grab local element coordinates.
        #--------------------------------
        self.xi   = self.points[:,0]
        self.eta  = self.points[:,1]
        self.zeta = self.points[:,2]
        #---------
        # Set N_a.
        #---------
        self.N1 = (1 - self.xi)*(1 - self.eta)*(1 - self.zeta)/8
        self.N2 = (1 + self.xi)*(1 - self.eta)*(1 - self.zeta)/8
        self.N3 = (1 + self.xi)*(1 + self.eta)*(1 - self.zeta)/8
        self.N4 = (1 - self.xi)*(1 + self.eta)*(1 - self.zeta)/8
        self.N5 = (1 - self.xi)*(1 - self.eta)*(1 + self.zeta)/8
        self.N6 = (1 + self.xi)*(1 - self.eta)*(1 + self.zeta)/8
        self.N7 = (1 + self.xi)*(1 + self.eta)*(1 + self.zeta)/8
        self.N8 = (1 - self.xi)*(1 + self.eta)*(1 + self.zeta)/8
        #-----------------------------
        # Build shape function matrix.
        #-----------------------------
        self.Nu = np.zeros((8, 3, 24), dtype=Parameters.float_dtype)
        for i in range(3):
            self.Nu[:, i, 0 + i]  = self.N1
            self.Nu[:, i, 3 + i]  = self.N2
            self.Nu[:, i, 6 + i]  = self.N3
            self.Nu[:, i, 9 + i]  = self.N4
            self.Nu[:, i, 12 + i] = self.N5
            self.Nu[:, i, 15 + i] = self.N6
            self.Nu[:, i, 18 + i] = self.N7
            self.Nu[:, i, 21 + i] = self.N8
```

In the above `self` refers to the element object (i.e., referencing itself).

In [44]:
print(element.Nu[0,:,:])

[[0.49056 0.      0.      0.13145 0.      0.      0.03522 0.      0.
  0.13145 0.      0.      0.13145 0.      0.      0.03522 0.      0.
  0.00944 0.      0.      0.03522 0.      0.     ]
 [0.      0.49056 0.      0.      0.13145 0.      0.      0.03522 0.
  0.      0.13145 0.      0.      0.13145 0.      0.      0.03522 0.
  0.      0.00944 0.      0.      0.03522 0.     ]
 [0.      0.      0.49056 0.      0.      0.13145 0.      0.      0.03522
  0.      0.      0.13145 0.      0.      0.13145 0.      0.      0.03522
  0.      0.      0.00944 0.      0.      0.03522]]


Recall the strain displacement matrix,

$$
B^{u,e} :=  \sum_{a=1}^8 \underbrace{B^u_a(\boldsymbol{\xi})}_{(n_{dim}\times n_{dim})\times(n_{dim}\times n_{Gauss})} = \sum_{a=1}^8 \underbrace{B^u_a(\boldsymbol{\xi})}_{9\times 24} = \sum_{a=1}^8 \frac{\partial\mathbf{N}_a^u}{\partial\boldsymbol{X}}
 = \sum_{a=1}^8 \frac{\partial\mathbf{N}_a^u}{\partial\boldsymbol{\xi}}\frac{\partial\boldsymbol{\xi}}{\partial\mathbf{X}}
$$

In [49]:
print(element.Bu.shape)

(8, 9, 24)


This portion of the code calculates $\dfrac{\partial\mathbf{N}_a^u}{\partial\boldsymbol{\xi}}$

```python
#----------------------------------
# Calculate derivatives w.r.t. \xi.
#----------------------------------
self.dN1_dxi = -(1/8)*(1 - self.eta)*(1 - self.zeta)
self.dN2_dxi = -self.dN1_dxi
self.dN3_dxi = (1/8)*(1 + self.eta)*(1 - self.zeta)
self.dN4_dxi = -self.dN3_dxi
self.dN5_dxi = -(1/8)*(1 - self.eta)*(1 + self.zeta)
self.dN6_dxi = -self.dN5_dxi
self.dN7_dxi = (1/8)*(1 + self.eta)*(1 + self.zeta)
self.dN8_dxi = -self.dN7_dxi

self.dN_dxi      = np.zeros((8,8), dtype=Parameters.float_dtype)
self.dN_dxi[:,0] = self.dN1_dxi
self.dN_dxi[:,1] = self.dN2_dxi
self.dN_dxi[:,2] = self.dN3_dxi
self.dN_dxi[:,3] = self.dN4_dxi
self.dN_dxi[:,4] = self.dN5_dxi
self.dN_dxi[:,5] = self.dN6_dxi
self.dN_dxi[:,6] = self.dN7_dxi
self.dN_dxi[:,7] = self.dN8_dxi
#-----------------------------------
# Calculate derivatives w.r.t. \eta.
#-----------------------------------
self.dN1_deta = -(1/8)*(1 - self.xi)*(1 - self.zeta)
self.dN2_deta = -(1/8)*(1 + self.xi)*(1 - self.zeta)
self.dN3_deta = -self.dN2_deta
self.dN4_deta = -self.dN1_deta
self.dN5_deta = -(1/8)*(1 - self.xi)*(1 + self.zeta)
self.dN6_deta = -(1/8)*(1 + self.xi)*(1 + self.zeta)
self.dN7_deta = -self.dN6_deta
self.dN8_deta = -self.dN5_deta

self.dN_deta      = np.zeros((8,8), dtype=Parameters.float_dtype)
self.dN_deta[:,0] = self.dN1_deta
self.dN_deta[:,1] = self.dN2_deta
self.dN_deta[:,2] = self.dN3_deta
self.dN_deta[:,3] = self.dN4_deta
self.dN_deta[:,4] = self.dN5_deta
self.dN_deta[:,5] = self.dN6_deta
self.dN_deta[:,6] = self.dN7_deta
self.dN_deta[:,7] = self.dN8_deta
#------------------------------------
# Calculate derivatives w.r.t. \zeta.
#------------------------------------
self.dN1_dzeta = -(1/8)*(1 - self.xi)*(1 - self.eta)
self.dN2_dzeta = -(1/8)*(1 + self.xi)*(1 - self.eta)
self.dN3_dzeta = -(1/8)*(1 + self.xi)*(1 + self.eta)
self.dN4_dzeta = -(1/8)*(1 - self.xi)*(1 + self.eta)
self.dN5_dzeta = -self.dN1_dzeta
self.dN6_dzeta = -self.dN2_dzeta
self.dN7_dzeta = -self.dN3_dzeta
self.dN8_dzeta = -self.dN4_dzeta

self.dN_dzeta      = np.zeros((8,8), dtype=Parameters.float_dtype)
self.dN_dzeta[:,0] = self.dN1_dzeta
self.dN_dzeta[:,1] = self.dN2_dzeta
self.dN_dzeta[:,2] = self.dN3_dzeta
self.dN_dzeta[:,3] = self.dN4_dzeta
self.dN_dzeta[:,4] = self.dN5_dzeta
self.dN_dzeta[:,5] = self.dN6_dzeta
self.dN_dzeta[:,6] = self.dN7_dzeta
self.dN_dzeta[:,7] = self.dN8_dzeta
#-------------------
# Compute jacobians.
#-------------------
self.get_Jacobian(Parameters)
```

This section of the code calculates $\dfrac{\partial\boldsymbol{X}}{\partial\boldsymbol{\xi}} = J^e$ or $\dfrac{\partial X_I}{\partial\xi_b} = J^e_{Ib}$, $\dfrac{\partial\boldsymbol{\xi}}{\partial\mathbf{X}} = (J^e)^{-1}$ or $\dfrac{\partial\xi_b}{\partial X_I} = J^e_{bI}$ (mapping of gradient) and $j^e = \det{J^e}$ (deformation mapping of element volume).

The ellipses, `...`, are an indicator to `np.einsum` that we do not want it to perform a summation over that axes, which is the axes of the 8 Gauss points. The first line means of the function below means compute $\dfrac{\partial x}{\partial\xi} = \dfrac{\partial N_a}{\partial\xi}x$ for all 8 Gauss points. And similarly the second line means compute $\dfrac{\partial x}{\partial\eta} = \dfrac{\partial N_a}{\partial\eta}x$ for all 8 Gauss points.

```python
def get_Jacobian(self, Parameters):
    # Compute the element Jacobian.
    self.dx_dxi   = np.einsum('...i, i -> ...', self.dN_dxi,   self.coordinates[:,0])
    self.dx_deta  = np.einsum('...i, i -> ...', self.dN_deta,  self.coordinates[:,0])
    self.dx_dzeta = np.einsum('...i, i -> ...', self.dN_dzeta, self.coordinates[:,0])

    self.dy_dxi   = np.einsum('...i, i -> ...', self.dN_dxi,   self.coordinates[:,1])
    self.dy_deta  = np.einsum('...i, i -> ...', self.dN_deta,  self.coordinates[:,1])
    self.dy_dzeta = np.einsum('...i, i -> ...', self.dN_dzeta, self.coordinates[:,1])

    self.dz_dxi   = np.einsum('...i, i -> ...', self.dN_dxi,   self.coordinates[:,2])
    self.dz_deta  = np.einsum('...i, i -> ...', self.dN_deta,  self.coordinates[:,2])
    self.dz_dzeta = np.einsum('...i, i -> ...', self.dN_dzeta, self.coordinates[:,2])

    self.Je        = np.zeros((8,3,3), dtype=Parameters.float_dtype)
    self.Je[:,0,0] = self.dx_dxi
    self.Je[:,0,1] = self.dx_deta
    self.Je[:,0,2] = self.dx_dzeta
    self.Je[:,1,0] = self.dy_dxi
    self.Je[:,1,1] = self.dy_deta
    self.Je[:,1,2] = self.dy_dzeta
    self.Je[:,2,0] = self.dz_dxi
    self.Je[:,2,1] = self.dz_deta
    self.Je[:,2,2] = self.dz_dzeta

    self.j     = np.zeros(8, dtype=Parameters.float_dtype)
    self.Jeinv = np.zeros((8,3,3), dtype=Parameters.float_dtype)

    for i in range(8):
        self.j[i]          = np.linalg.det(self.Je[i,:,:])
        self.Jeinv[i,:,:,] = np.linalg.inv(self.Je[i,:,:])

    return
```

Now that we have the gradient mapping, we can construct the strain displacement matrix as follows, where the first few lines are the operation $\dfrac{\partial\mathbf{N}_a^u}{\partial\boldsymbol{\xi}}\dfrac{\partial\boldsymbol{\xi}}{\partial\mathbf{X}}$, or, $\dfrac{\partial N_a^u}{\partial\xi_b}\dfrac{\partial\xi_b}{\partial X_I} = \dfrac{\partial N_a^u}{\partial X_I}$.

Again, the ellipses tell `np.einsum` not to sum over the 8 Gauss points.

```python
# Continued evaluate_Shape_Functions from above...
#----------------------------------
# Compute shape function gradients.
#----------------------------------
self.dN1_dx = np.einsum('...bI, ...b -> ...I', self.Jeinv, np.array([self.dN1_dxi, self.dN1_deta, self.dN1_dzeta]).T)
self.dN2_dx = np.einsum('...bI, ...b -> ...I', self.Jeinv, np.array([self.dN2_dxi, self.dN2_deta, self.dN2_dzeta]).T)
self.dN3_dx = np.einsum('...bI, ...b -> ...I', self.Jeinv, np.array([self.dN3_dxi, self.dN3_deta, self.dN3_dzeta]).T)
self.dN4_dx = np.einsum('...bI, ...b -> ...I', self.Jeinv, np.array([self.dN4_dxi, self.dN4_deta, self.dN4_dzeta]).T)
self.dN5_dx = np.einsum('...bI, ...b -> ...I', self.Jeinv, np.array([self.dN5_dxi, self.dN5_deta, self.dN5_dzeta]).T)
self.dN6_dx = np.einsum('...bI, ...b -> ...I', self.Jeinv, np.array([self.dN6_dxi, self.dN6_deta, self.dN6_dzeta]).T)
self.dN7_dx = np.einsum('...bI, ...b -> ...I', self.Jeinv, np.array([self.dN7_dxi, self.dN7_deta, self.dN7_dzeta]).T)
self.dN8_dx = np.einsum('...bI, ...b -> ...I', self.Jeinv, np.array([self.dN8_dxi, self.dN8_deta, self.dN8_dzeta]).T)
#--------------------------------------
# Construct strain-displacement matrix.
#--------------------------------------
self.Bu = np.zeros((8, 9, 24), dtype=Parameters.float_dtype)
# These for-loops just stagger where the gradients are put in the rows of Bu.
# Refer to MATLAB code which allows array definition with better visualization
# than Python.
for i in range(3):
    self.Bu[:, i, 0]  = self.dN1_dx[:,i]
    self.Bu[:, i, 3]  = self.dN2_dx[:,i]
    self.Bu[:, i, 6]  = self.dN3_dx[:,i]
    self.Bu[:, i, 9]  = self.dN4_dx[:,i]
    self.Bu[:, i, 12] = self.dN5_dx[:,i]
    self.Bu[:, i, 15] = self.dN6_dx[:,i]
    self.Bu[:, i, 18] = self.dN7_dx[:,i]
    self.Bu[:, i, 21] = self.dN8_dx[:,i]

for i in range(3,6):
    self.Bu[:, i, 1]  = self.dN1_dx[:,i-3]
    self.Bu[:, i, 4]  = self.dN2_dx[:,i-3]
    self.Bu[:, i, 7]  = self.dN3_dx[:,i-3]
    self.Bu[:, i, 10] = self.dN4_dx[:,i-3]
    self.Bu[:, i, 13] = self.dN5_dx[:,i-3]
    self.Bu[:, i, 16] = self.dN6_dx[:,i-3]
    self.Bu[:, i, 19] = self.dN7_dx[:,i-3]
    self.Bu[:, i, 22] = self.dN8_dx[:,i-3]

for i in range(6,9):
    self.Bu[:, i, 2]  = self.dN1_dx[:,i-6]
    self.Bu[:, i, 5]  = self.dN2_dx[:,i-6]
    self.Bu[:, i, 8]  = self.dN3_dx[:,i-6]
    self.Bu[:, i, 11] = self.dN4_dx[:,i-6]
    self.Bu[:, i, 14] = self.dN5_dx[:,i-6]
    self.Bu[:, i, 17] = self.dN6_dx[:,i-6]
    self.Bu[:, i, 20] = self.dN7_dx[:,i-6]
    self.Bu[:, i, 23] = self.dN8_dx[:,i-6]

return
```

Now that we have our interpolation functions, we can compute the local strains and stresses, e.g.,

(`@register_method` is a wrapper function that allows us to access object methods from other files so that we do not have to define all of the class methods in one file; these methods are defined in `_ElementVariables.py` and not in `classElement.py`)

```python
@register_method
def get_dudX(self, Parameters):
    # Compute solid displacement gradient.
    self.dudX = np.einsum('...ij, j -> ...i', self.Bu, self.u_global, dtype=Parameters.float_dtype)
    return
    
@register_method
def get_F(self, Parameters):
    # Compute deformation gradient.
    #----------------------------------------------------
    # Reshape the identity matrix for all 8 Gauss points.
    #----------------------------------------------------
    self.identity = np.zeros((8,3,3), dtype=Parameters.float_dtype)
    np.einsum('ijj -> ij', self.identity)[:] = 1
    #-------------------------------------------------------
    # Create the 3x3 deformation matrix from the 9x1 vector.
    #-------------------------------------------------------
    self.dudX_mat = np.zeros((8,3,3), dtype=Parameters.float_dtype)
    for i in range(9):
        if i == 0:
            alpha = 0
            beta  = 0
        elif i == 1:
            alpha = 0
            beta  = 1
        elif i == 2:
            alpha = 0
            beta  = 2
        elif i == 3:
            alpha = 1
            beta  = 0
        elif i == 4:
            alpha = 1
            beta  = 1
        elif i == 5:
            alpha = 1
            beta  = 2
        elif i == 6:
            alpha = 2
            beta  = 0
        elif i == 7:
            alpha = 2
            beta  = 1
        elif i == 8:
            alpha = 2
            beta  = 2
        self.dudX_mat[:,alpha,beta] = self.dudX[:,i]

    self.F = self.identity + self.dudX_mat
    return

@register_method
def get_F_inv(self):
    # Compute inverse of deformation gradient.
    self.F_inv = np.linalg.inv(self.F)
    return

@register_method
def get_J(self):
    # Compute Jacobian of deformation.
    self.J = np.linalg.det(self.F)
    return

@register_method
def get_C(self, Parameters):
    # Compute right Cauchy-Green tensor.
    self.C = np.einsum('...iI, ...iJ -> ...IJ', self.F, self.F, dtype=Parameters.float_dtype)
    return

@register_method
def get_C_inv(self):
    # Compute inverse of right Cauchy-Green tensor.
    self.C_inv = np.linalg.inv(self.C)
    return

@register_method
def get_SPK(self, Parameters):
    # Compute second Piola-Kirchoff stress tensor.
    self.SPK = Parameters.mu*self.identity + np.einsum('..., ...IJ -> ...IJ',\
                                                        Parameters.lambd*np.log(self.J) - Parameters.mu,\
                                                        self.C_inv, dtype=Parameters.float_dtype)
    return

@register_method
def get_FPK(self, Parameters):
    # Compute first Piola-Kirchoff stress tensor.
    self.FPK = np.einsum('...iI, ...IJ -> ...iI', self.F, self.SPK, dtype=Parameters.float_dtype)
    return
```

Below are the internal and external force vectors $\mathcal{G}_1^{\text{INT},h^e}$, $\mathcal{G}_2^{\text{INT},h^e}$ and $\mathcal{G}^{\text{EXT},h^e}$. Here there is an explicit summation over the axis that represents the 8 Gauss points, i.e., a sum over the Gauss point index $k$.

$\underbrace{\mathcal{G}_1^{\text{INT},h^e}}_{24\times 1} = \int\limits_{B_0^e}\underbrace{\mathbf{B}^{e^T}}_{24\times 9}\underbrace{\mathbf{P}^{h^e}}_{9\times 1}dV$

where $\mathbf{P}^{h^e}$ is expressed in Voigt notation $\underbrace{P_\alpha^{h^e}}_{9\times 1}$ from $\underbrace{P_{iI}^{h^e}}_{3\times 3}$ (remember Python is 0-based indexing, so 00 means 11 if we were to translate to handwritten matrix notation, and similarly 02 means 13).

```python
@register_method
def get_G1(self, Parameters):
    # Compute G_1^INT.
    self.FPK_voigt = np.zeros((8,9), dtype=Parameters.float_dtype)
    for alpha in range(9):
        if alpha == 0:
            i = 0
            I = 0
        elif alpha == 1:
            i = 0
            I = 1
        elif alpha == 2:
            i = 0
            I = 2
        elif alpha == 3:
            i = 1
            I = 0
        elif alpha == 4:
            i = 1
            I = 1
        elif alpha == 5:
            i = 1
            I = 2
        elif alpha == 6:
            i = 2
            I = 0
        elif alpha == 7:
            i = 2
            I = 1
        elif alpha == 8:
            i = 2
            I = 2
        self.FPK_voigt[:,alpha] = self.FPK[:,i,I]

    self.G_1 = np.einsum('kij, ki, k -> j', self.Bu, self.FPK_voigt, self.weights*self.j)
    return
```

$\underbrace{\mathcal{G}_2^{\text{INT},h^e}}_{24\times 1} = -\int\limits_{B_0^e}\cdot\underbrace{\mathbf{N}^{e^T}}_{24\times 3}\underbrace{\rho\mathbf{g}}_{3\times 1}dV$

```python
@register_method
def get_G2(self, Parameters):
    # Compute G_2^INT.
    self.grav_body       = np.zeros((8,3), dtype=Parameters.float_dtype)
    self.grav_body[:,2]  = -Parameters.grav
    
    self.G_2 = np.einsum('kij, ki, k -> j', -self.Nu, self.rho_0*self.grav_body, self.weights*self.j)
    return
```

$\underbrace{\mathcal{G}^{\text{EXT},h^e}}_{24\times 1} = \int\limits_{\Gamma_0^e}\underbrace{\mathbf{N}^{e^T}}_{24\times 3}\underbrace{\mathbf{t}^\sigma}_{3\times 1}dA$

```python
@register_method
def get_GEXT(self, Parameters):
    # Compute G^EXT (for topmost element only).
    if self.ID == (Parameters.numEl - 1) and Parameters.tractionProblem:
        self.traction      = np.zeros((4,3), dtype=Parameters.float_dtype)
        self.traction[:,2] = -Parameters.traction
    
        self.evaluate_Shape_Functions_2D(Parameters)
        self.G_EXT = np.einsum('kij, ki, k -> j', self.Nu_2D, self.traction, self.weights[4:8]*self.j_2D)
    else:
        self.G_EXT = np.zeros((24), dtype=Parameters.float_dtype)
    return
```

Note in the last function that the traction can only be applied to the top element, and that we need to compute the interpolation related to the plane-element (the element boundary at the top surface). Thus in the below function, the shape functions are only evaluated for $\zeta = 1$ and thus $N_1, N_2, N_3, N_4 = 0$.

```python
def evaluate_Shape_Functions_2D(self, Parameters):
    # Create a 2D planar element for traction boundary condition.
    #---------
    # Set N_a.
    #---------
    self.N5_2D = (1 - self.xi[4:8])*(1 - self.eta[4:8])/4
    self.N6_2D = (1 + self.xi[4:8])*(1 - self.eta[4:8])/4
    self.N7_2D = (1 + self.xi[4:8])*(1 + self.eta[4:8])/4
    self.N8_2D = (1 - self.xi[4:8])*(1 + self.eta[4:8])/4
    #-----------------------------
    # Build shape function matrix.
    #-----------------------------
    self.Nu_2D = np.zeros((4, 3, 24), dtype=Parameters.float_dtype)
    for i in range(3):
        self.Nu_2D[:, i, 12 + i] = self.N5_2D
        self.Nu_2D[:, i, 15 + i] = self.N6_2D
        self.Nu_2D[:, i, 18 + i] = self.N7_2D
        self.Nu_2D[:, i, 21 + i] = self.N8_2D
    #----------------------------------
    # Calculate derivatives w.r.t. \xi.
    #----------------------------------
    self.dN5_dxi_2D = -(1/4)*(1 - self.eta[4:8])
    self.dN6_dxi_2D = -self.dN5_dxi_2D
    self.dN7_dxi_2D = (1/4)*(1 + self.eta[4:8])
    self.dN8_dxi_2D = -self.dN7_dxi_2D

    self.dN_dxi_2D      = np.zeros((4,4), dtype=Parameters.float_dtype)
    self.dN_dxi_2D[:,0] = self.dN5_dxi_2D
    self.dN_dxi_2D[:,1] = self.dN6_dxi_2D
    self.dN_dxi_2D[:,2] = self.dN7_dxi_2D
    self.dN_dxi_2D[:,3] = self.dN8_dxi_2D
    #-----------------------------------
    # Calculate derivatives w.r.t. \eta.
    #-----------------------------------
    self.dN5_deta_2D = -(1/4)*(1 - self.xi[4:8])
    self.dN6_deta_2D = -(1/4)*(1 + self.xi[4:8])
    self.dN7_deta_2D = -self.dN6_deta_2D
    self.dN8_deta_2D = -self.dN5_deta_2D

    self.dN_deta_2D      = np.zeros((4,4), dtype=Parameters.float_dtype)
    self.dN_deta_2D[:,0] = self.dN5_deta_2D
    self.dN_deta_2D[:,1] = self.dN6_deta_2D
    self.dN_deta_2D[:,2] = self.dN7_deta_2D
    self.dN_deta_2D[:,3] = self.dN8_deta_2D
    #------------------------
    # Calculate the jacobian.
    #------------------------
    self.get_Jacobian_2D(Parameters)

    return

def get_Jacobian_2D(self, Parameters):
    # Compute the 2D element Jacobian.
    self.dx_dxi_2D  = np.einsum('...i, i -> ...', self.dN_dxi_2D,   self.coordinates[4:8,0])
    self.dx_deta_2D = np.einsum('...i, i -> ...', self.dN_deta_2D,  self.coordinates[4:8,0])

    self.dy_dxi_2D  = np.einsum('...i, i -> ...', self.dN_dxi_2D,   self.coordinates[4:8,1])
    self.dy_deta_2D = np.einsum('...i, i -> ...', self.dN_deta_2D,  self.coordinates[4:8,1])

    self.Je_2D        = np.zeros((4,2,2), dtype=Parameters.float_dtype)
    self.Je_2D[:,0,0] = self.dx_dxi_2D
    self.Je_2D[:,0,1] = self.dx_deta_2D
    self.Je_2D[:,1,0] = self.dy_dxi_2D
    self.Je_2D[:,1,1] = self.dy_deta_2D

    self.j_2D = np.zeros(4, dtype=Parameters.float_dtype)

    for i in range(4):
        self.j_2D[i] = np.linalg.det(self.Je_2D[i,:,:])

    return
```

We also need the consistent tangent $\mathcal{G}_{u,u}^{h^e}$, defined below.

$$
\delta\mathcal{G}^{h^e}_1 = \int\limits_{B_0^e}\Bigg[\frac{\partial w_i^{h^e}}{\partial X_I}\Bigg]\Bigg[\frac{\partial P_{iI}}{\partial F_{aA}}\Bigg]^{h^e}\Bigg[\frac{\partial(\delta u_a^{h^e})}{\partial X_A}\Bigg]dV
$$

Note the indices matching with the mathematical formulation,

$$
\dfrac{\partial P_{iI}}{\partial F_{aA}} = \delta_{ai}S_{AI} + \lambda F_{Aa}^{-1}F_{Ii}^{-1} - (\lambda\log(J) - \mu)(F_{Ai}^{-1}F_{Ia}^{-1} + \delta_{ai}C_{AI}^{-1})
$$

which is a 3x3x3x3 (or 8x3x3x3x3 in the code, for the 8 Gauss points). In order to correctly compute the tangent, we must transform this to a 9x9 (or 8x9x9 in the code, for the 8 Gauss points) using Voigt notation, i.e.,

$$
\dfrac{\partial P_{iI}}{\partial F_{aA}} \rightarrow D_{\alpha,\beta}
$$ 

```python
@register_method
def get_G_uu_1(self, Parameters):
    # Compute G_uu_1.
    self.dPdF = np.einsum('...ai, ...AI -> ...iIaA', self.identity, self.SPK)\
                    + Parameters.lambd*np.einsum('...Aa, ...Ii -> ...iIaA', self.F_inv, self.F_inv)\
                    - np.einsum('..., ...iIaA -> ...iIaA', Parameters.lambd*np.log(self.J) - Parameters.mu,\
                                (np.einsum('...Ai, ...Ia -> ...iIaA', self.F_inv, self.F_inv)\
                                + np.einsum('...ai, ...AI -> ...iIaA', self.identity, self.C_inv)))

    self.dPdF_voigt = np.zeros((8,9,9), dtype=Parameters.float_dtype)
    for alpha in range(9):
        if alpha == 0:
            i = 0
            I = 0
        elif alpha == 1:
            i = 0
            I = 1
        elif alpha == 2:
            i = 0
            I = 2
        elif alpha == 3:
            i = 1
            I = 0
        elif alpha == 4:
            i = 1
            I = 1
        elif alpha == 5:
            i = 1
            I = 2
        elif alpha == 6:
            i = 2
            I = 0
        elif alpha == 7:
            i = 2
            I = 1
        elif alpha == 8:
            i = 2
            I = 2
        for beta in range(9):
            if beta == 0:
                a = 0
                A = 0
            elif beta == 1:
                a = 0
                A = 1
            elif beta == 2:
                a = 0
                A = 2
            elif beta == 3:
                a = 1
                A = 0
            elif beta == 4:
                a = 1
                A = 1
            elif beta == 5:
                a = 1
                A = 2
            elif beta == 6:
                a = 2
                A = 0
            elif beta == 7:
                a = 2
                A = 1
            elif beta == 8:
                a = 2
                A = 2

            self.dPdF_voigt[:,alpha,beta] = self.dPdF[:,i,I,a,A]

    self.G_uu_1 = np.einsum('kiI, kij, kjJ, k -> IJ', self.Bu, self.dPdF_voigt, self.Bu, self.weights*self.j)
    return
```